# 데이터 전처리

## Import Main File

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

path = '/home/piai/포스코AI아카데미/BigData/1. 데이터셋/Project/01_tags2/'
df = pd.DataFrame()

for i in range(1, 21): 
    tmp = pd.read_csv(path+'tags_%d.csv'%i)
    if df.empty:
        df = tmp
    else:
        df = df.append(tmp)
print('Finished !! ')
df_pivot = pd.pivot_table(data = df, index = 'datetime', columns = 'id')

Missing colon in file PosixPath('/home/piai/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/matplotlibrc'), line 748 ('font.f')
Missing colon in file '/home/piai/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/matplotlibrc', line 748 ('font.f')


## Pivotting Main DataFrame

In [4]:
pivot_df = pd.pivot_table(data = df, index = 'datetime', columns = 'id')

In [168]:
pivot_df.index = pd.to_datetime(pivot_df.index)
pivot_df.columns = [col[1] for col in pivot_df.columns]

## Export csv

In [181]:
pivot_df.to_csv('./1. DataSet/pivotdata.csv', index = True)

## Import csv

In [166]:
process_dict = {1:["TAG_{:003d}".format(i) for i in range(1,57)],2:["TAG_{:003d}".format(i) for i in range(57,72)],3:["TAG_{:003d}".format(i) for i in range(72,87)],4:["TAG_{:003d}".format(i) for i in range(87,196)]}
pivot_df = pd.read_csv("../1. DataSet/pivotdata.csv", index_col = 'Unnamed: 0')
pivot_df.index = pd.to_datetime(pivot_df.index)
df_info = pd.read_csv("../1. DataSet/tag_list.csv", encoding = 'euc-kr' )
df_stop_list = pd.read_csv("../1. DataSet/03_중지리스트.csv")

## Fill up target variable

In [167]:
df_stop_list = pd.read_csv("../1. DataSet/03_중지리스트.csv")
df_indexes =  {time : 0 for time in pivot_df.index}
problems = df_stop_list[df_stop_list['cause'].isin(['중지', '불량중지'])]
etcs = df_stop_list[~df_stop_list['cause'].isin(['중지', '불량중지'])]

## A case for Multinomial Distribution.
# for start, end in zip(pd.to_datetime(etcs['start_date']), pd.to_datetime(etcs['end_date'])):
#     basetime = start
#     for time in range((end - start).seconds // 60):
#         df_indexes[ basetime + datetime.timedelta(minutes = time)] = 2

for start, end in zip(pd.to_datetime(problems['start_date']), pd.to_datetime(problems['end_date'])):
    basetime = start
    for time in range((end - start).seconds // 60):
        df_indexes[ basetime + datetime.timedelta(minutes = time)] = 1

pivot_df['target'] = [df_indexes[ind] for ind in pivot_df.index]

In [4]:
n = 7 # 몇 개끼리 뭉쳐서 자를 거냐
pivot_df.reset_index(inplace =True)
tmp = pivot_df.columns
tmp = list(tmp)
tmp[0] = 'datetime'
pivot_df.columns = tmp
pivot_tmp = pivot_df[:]
pivot_tmp['target'] = pivot_df['target']
pivot_tmp['result'] = (pivot_tmp['target'] == 1).astype(int)
pivot_tmp['result'] = pivot_tmp['result'].shift(-n)
pivot_tmp = pivot_tmp.loc[:len(pivot_tmp)-n-1] 

In [9]:
# rided_data = pivot_tmp[pivot_tmp['target'] != 0] # 0이 아닌 놈들 지우기
rided_data = pivot_tmp[pivot_tmp['target'] != 0] # 0이 아닌 놈들 지우기, 정상 생산인 놈들만 select
indexes = list(pivot_tmp.index)

for ind in rided_data.index:
    indexes[ind] = 0

indexes = list(map(str, indexes))
indexes = '/'.join(indexes)
indexes = indexes.replace('/0', '/')
indexes = indexes.split('//')
indexes = [ind for ind in indexes if ind]
indexes = [ind if ind[0] != '/' else ind[1:] for ind in indexes]
indexes = [[int(ind.split('/')[0]), int(ind.split('/')[-1])] for ind in indexes if ind]

In [ ]:
train_df = pd.DataFrame()
for from_val, to_val in indexes:
    result_df = pd.DataFrame()
    for ind in range(from_val+n, to_val+1, n):
        tmp = pivot_tmp.loc[ind-n:ind]
        tmp_datetime = tmp['datetime'].values[0]
        tmp = pd.DataFrame(tmp.mean()).T
        tmp['datetime'] = tmp_datetime
        tmp['result'] = tmp['result'].astype(bool)
        tmp['result'] = tmp['result'].astype(int)
        if result_df.empty:
            result_df = tmp
        else:
            result_df = result_df.append(tmp, ignore_index=True)
    if train_df.empty:
        train_df = result_df
    else:
        train_df = train_df.append(result_df, ignore_index=True)

## Make train data sets

In [230]:
def makedf(pivot_df, df_stop, split = 7, interval = 30, removal = 120, multi = False):
    # 중지 리스트, 피벗 테이블
    # 피벗테이블 2 가지 변수 1 : origin_result 2 : result
    # 불량 중지 리스트에서  읽어들여  1. start 2. end + removal 만큼 true 값 만들어 indexes 리스트 생성
    # indexes 돌려서 제거
    print('1 Preprocessing start : 0%')
    
    if 'datetime' in pivot_df.columns:
        pivot_df.index = pivot_df['datetime']
        pivot_df.drop('datetime', axis = 1, inplace = True)
    
    dic_indexes =  {time : ind for ind, time in enumerate(pivot_df.index)}
    pivot_df['result'] = [0] * len(pivot_df)
    problems = df_stop_list[df_stop_list['cause'].isin(['중지', '불량중지'])] # 고장 Labeling을 위함.
    etcs = df_stop_list[~df_stop_list['cause'].isin(['중지', '불량중지'])] # 기타 Labeling을 위함.
    print('1 Preprocessing. 완료\n2 time setting start. : 20%')
    
    result = pivot_df['result']            
    for start, end in zip(pd.to_datetime(problems['start_date']), pd.to_datetime(problems['end_date'])):
        basetime = start
        for inter in range(interval):
            result.loc[basetime - datetime.timedelta(minutes =  inter)] = (interval - inter)
        for time in range((end - start).seconds // 60 + removal):
            dic_indexes[ basetime + datetime.timedelta(minutes = time)] = 0  # 잘라 없앨 것들
            
    if 'datetime' not in pivot_df.columns:
        pivot_df.reset_index(inplace = True)
        tmp = pivot_df.columns
        tmp = list(tmp)
        tmp[0] = 'datetime'
        pivot_df.columns = tmp
        
    indexes = list(map(str, dic_indexes.values()))
    indexes = '/'.join(indexes)
    indexes = indexes.replace('/0', '/')
    indexes = indexes.split('//')
    indexes = [ind for ind in indexes if ind]
    indexes = [ind if ind[0] != '/' else ind[1:] for ind in indexes]
    indexes = [[int(ind.split('/')[0]), int(ind.split('/')[-1])] for ind in indexes if ind]
    
    group_val = 1
    group_list = [0] * len(pivot_df)
    for start, end in indexes:
        for ind in range(start, end + 1, split):
            for sp in range(split):
                if ind + sp > end:
                    break
                if pivot_df.loc[ind+sp]['result']:
                    group_val += 1
                group_list[ind + sp] = group_val
            group_val += 1
    
    pivot_df['group'] = group_list # group이 0인 것 삭제 
    pivot_df = pivot_df[pivot_df['group'] != 0]    
    tmp_tag, tmp_else =pivot_df.select_dtypes(include = 'float'), pivot_df.select_dtypes(exclude = 'float')
    tmp_tag['group'] = pivot_df['group']
    tmp_tag = tmp_tag.groupby('group', as_index = False).mean()
    tmp_else = tmp_else.groupby('group', as_index = False).max()
    df_result = pd.concat([tmp_tag, tmp_else], axis = 1)
    df_result.drop('group', inplace = True, axis = 1)
    df_result

    return df_result

In [232]:
splits = [5, 10, 20, 30]
intervals = [10, 30, 60, 120]
removals = [30, 60, 120, 240]
for split in splits[::-1]:
    for interval in intervals[::-1]:
        for removal in removals[::-1]:
            df_result = makedf(pivot_df, df_stop_list, split = split, interval = interval, removal = removal)
            df_result.to_csv('sp_%d__inter_%d__rm_%d.csv'%(split, interval, removal))

1 Preprocessing start : 0%
1 Preprocessing. 완료
2 time setting start. : 20%


/home/piai/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_30741/2171133328.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_tag['group'] = pivot_df['group']


1 Preprocessing start : 0%
1 Preprocessing. 완료
2 time setting start. : 20%


/home/piai/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_30741/2171133328.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_tag['group'] = pivot_df['group']


1 Preprocessing start : 0%
1 Preprocessing. 완료
2 time setting start. : 20%


...

/home/piai/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_30741/2171133328.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_tag['group'] = pivot_df['group']


1 Preprocessing start : 0%
1 Preprocessing. 완료
2 time setting start. : 20%


/home/piai/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_30741/2171133328.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_tag['group'] = pivot_df['group']


1 Preprocessing start : 0%
1 Preprocessing. 완료
2 time setting start. : 20%


/home/piai/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_30741/2171133328.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_tag['group'] = pivot_df['group']
